In [12]:
print("hello")

hello


In [13]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [14]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [15]:
extracted_data = load_pdf_file(data=r'C:\Users\Rishabh\Desktop\medical-chatbotgenai\llmp\Data')


In [17]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [18]:
text_chunks=text_split(extracted_data)
print("Length of text chunks",len(text_chunks))

Length of text chunks 39994


In [19]:
pip install -U langchain-huggingface


Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain.embeddings import HuggingFaceEmbeddings


In [21]:
def downloading_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings=downloading_hugging_face_embedding()

C:\Users\Rishabh\AppData\Local\Temp\ipykernel_11072\1414041956.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [22]:
quer

NameError: name 'quer' is not defined

In [23]:
embeddings=downloading_hugging_face_embedding()

In [24]:
query_result=embeddings.embed_query("hello world")
print("Length",len(query_result))

Length 384


In [25]:
from dotenv import load_dotenv
load_dotenv()

False

In [26]:
import os
PINECONE_API_KEY=os.environ.get('pcsk_4paSYZ_U58HJwaSWY5k39ZFXWGpCpwaPtgiadAJsGmHTb63AtqNkfWweDJwNioWPaSetKN')

In [41]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_4paSYZ_U58HJwaSWY5k39ZFXWGpCpwaPtgiadAJsGmHTb63AtqNkfWweDJwNioWPaSetKN")
index_name = "quickstart"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [42]:
import os
os.environ["PINECONE_API_KEY"]="pcsk_4paSYZ_U58HJwaSWY5k39ZFXWGpCpwaPtgiadAJsGmHTb63AtqNkfWweDJwNioWPaSetKN"

In [47]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,         # ✅ your list of text chunks
    embedding=embeddings,          # ✅ an embedding model, like OpenAIEmbeddings()
    index_name=index_name          # ✅ the name of your Pinecone index
)


In [50]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings  # ✅ use the correct keyword
)


In [51]:
docsearch

In [52]:
retriver=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [53]:
retrived_docs=retriver.invoke("what is Acne?")

In [54]:
retrived_docs

[Document(metadata={'page': 55.0, 'source': 'C:\\Users\\Rishabh\\Desktop\\medical-chatbotgenai\\llmp\\Data\\data.pdf'}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(metadata={'page': 55.0, 'source': 'C:\\Users\\Rishabh\\Desktop\\medical-chatbotgenai\\llmp\\Data\\data.pdf'}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(metadata={'page': 55.0, 'source': 'C:\\Users\\Rishabh\\Desktop\\medical-chatbotgenai\\llmp\\Data\\data.pdf'}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne')]

In [5]:
pip install gpt4all


   ---------------------------------------- 0.0/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/119.6 MB ? eta -:--:--
   ---------------------------------------- 0.5/119.6 MB 316.6 kB/s eta 0:06:17
   ---------------------------------------- 0.5/119.6 MB 316.6 kB/s 

In [ ]:
from langchain.llms import GPT4All

llm = GPT4All(model="C:\Users\Rishabh\Desktop\medical-chatbotgenai\llmp\model\gpt4all-installer-win64.exe", verbose=True)


FileNotFoundError: Model file does not exist: WindowsPath('C:/Users/Rishabh/.cache/gpt4all/llmp/model/gpt4all-installer-win64.exe.gguf')

In [3]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])


In [4]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriver,question_answer_chain)

NameError: name 'llm' is not defined

In [1]:
response = rag_chain.invoke({"input": "what is Acne"})
print(response["answer"])




NameError: name 'rag_chain' is not defined

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement deepseek_llm (from versions: none)
ERROR: No matching distribution found for deepseek_llm
